In [ ]:
import os
import sys
import shutil
import subprocess
from huggingface_hub import HfApi, login, snapshot_download

HF_TOKEN = "os.getenv("HF_TOKEN")"

# 변환할 원본 모델
SOURCE_MODEL_ID = "WindyAle/Kanana-Finance-v1" 

# 업로드할 리포지토리
TARGET_REPO_ID = "WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF" 

# 파일명 설정
GGUF_FILENAME_FP16 = "model-f16.gguf"
GGUF_FILENAME_Q4 = "model-Q4_K_M.gguf"

BASE_DIR = os.path.abspath(".")
MODEL_DIR = os.path.join(BASE_DIR, "downloaded_model")


LLAMA_CPP_DIR = os.path.join(BASE_DIR, "llama.cpp")


In [ ]:
def run_command(command, cwd=None):
    print(f"실행: {' '.join(command)}")
    try:
        subprocess.run(command, check=True, cwd=cwd)
    except subprocess.CalledProcessError as e:
        print(f"오류 발생: {e}")
        sys.exit(1)

In [ ]:
def load_model():
    if not HF_TOKEN:
        print("오류: HF_TOKEN 없음")
        sys.exit(1)
    login(token=HF_TOKEN)
    
    print(f"원본 모델: {SOURCE_MODEL_ID}")
    print(f"타겟 리포지토리: {TARGET_REPO_ID}")

    print(f"\n모델 다운로드: ({SOURCE_MODEL_ID})")
    if os.path.exists(MODEL_DIR):
        shutil.rmtree(MODEL_DIR)
        
    try:
        snapshot_download(
            repo_id=SOURCE_MODEL_ID,
            local_dir=MODEL_DIR,
            local_dir_use_symlinks=False,
            ignore_patterns=["*.gguf", "*.bin", "*.pth"]
        )
        print("✅ 다운로드 완료")
    except Exception as e:
        print(f"다운로드 실패: {e}")
        sys.exit(1)

In [ ]:


def set_llama_cpp():
    if not os.path.exists(LLAMA_CPP_DIR):
        run_command(["git", "clone", "https://github.com/ggerganov/llama.cpp.git"])
    
    # 필수 패키지 설치
    run_command([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], cwd=LLAMA_CPP_DIR)
    
    # CMake 빌드 (양자화 도구 생성)
    print("🔨 llama.cpp 빌드 중 (CMake)...")
    build_dir = os.path.join(LLAMA_CPP_DIR, "build")
    run_command(["cmake", "-B", build_dir, "-DGGML_NATIVE=OFF"], cwd=LLAMA_CPP_DIR)
    run_command(["cmake", "--build", build_dir, "--config", "Release", "-j", "4"], cwd=LLAMA_CPP_DIR)
    
    # 양자화 실행 파일 위치 찾기
    quantize_bin = os.path.join(build_dir, "bin", "llama-quantize")
    if not os.path.exists(quantize_bin):
        # 경로가 다를 경우 (루트 빌드 등) 대비
        quantize_bin = os.path.join(build_dir, "llama-quantize")
        
    if not os.path.exists(quantize_bin):
        print("❌ 오류: llama-quantize 바이너리를 찾을 수 없습니다. 빌드 실패.")
        sys.exit(1)

    # ------------------------------------------------------------------
    # 3. GGUF 변환 (FP16)
    # ------------------------------------------------------------------
    print(f"\n[3/5] GGUF 변환 (FP16) 수행 중...")
    convert_script = os.path.join(LLAMA_CPP_DIR, "convert_hf_to_gguf.py")
    
    run_command([
        sys.executable, convert_script,
        MODEL_DIR,
        "--outfile", GGUF_FILENAME_FP16,
        "--outtype", "f16"
    ])
    
    if not os.path.exists(GGUF_FILENAME_FP16):
        print("❌ 변환 실패: FP16 파일이 생성되지 않았습니다.")
        sys.exit(1)

    # ------------------------------------------------------------------
    # 4. 4bit 양자화 (Q4_K_M)
    # ------------------------------------------------------------------
    print(f"\n[4/5] 4bit 양자화 (Q4_K_M) 수행 중...")
    
    run_command([
        quantize_bin,
        GGUF_FILENAME_FP16,
        GGUF_FILENAME_Q4,
        "Q4_K_M"
    ])
    
    if not os.path.exists(GGUF_FILENAME_Q4):
        print("❌ 양자화 실패: Q4 파일이 생성되지 않았습니다.")
        sys.exit(1)
        
    print(f"✅ 양자화 완료: {GGUF_FILENAME_Q4}")
    
    # (선택사항) 용량 확보를 위해 FP16 파일 삭제
    os.remove(GGUF_FILENAME_FP16)
    shutil.rmtree(MODEL_DIR) 



if __name__ == "__main__":
    main()


=== 작업 시작 ===
원본 모델: WindyAle/Kanana-Finance-v1
타겟 리포지토리: WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF

[1/5] 모델 다운로드 중... (WindyAle/Kanana-Finance-v1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

✅ 다운로드 완료

[2/5] llama.cpp 준비 중...
🚀 실행: git clone https://github.com/ggerganov/llama.cpp.git


Cloning into 'llama.cpp'...
Updating files: 100% (2146/2146), done.


...

INFO:hf-to-gguf:Loading model: downloaded_model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {1792, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {1792}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {8064, 1792}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {1792, 8064}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {1792, 8064}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {1792}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {1792, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> F16, shape = {3072, 1792}
INFO:hf-to-gguf:blk.0.a


[4/5] 4bit 양자화 (Q4_K_M) 수행 중...
🚀 실행: /workspace/llama.cpp/build/bin/llama-quantize model-f16.gguf model-Q4_K_M.gguf Q4_K_M


main: build = 7807 (0e4ebeb05)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing 'model-f16.gguf' to 'model-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: direct I/O is enabled, disabling mmap
llama_model_loader: loaded meta data with 29 key-value pairs and 290 tensors from model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Downloaded_Model
llama_model_loader: - kv   3:                         general.size_label str              = 2.1B
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                       llama.context_length u32    


main: quantize time = 20525.37 ms
main:    total time = 20525.37 ms
✅ 양자화 완료: model-Q4_K_M.gguf

[5/5] Hugging Face 업로드 시작 (WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF)...


HfHubHTTPError: (Request ID: Root=1-697242d1-3c681d077c82d72c5baf5263;6433bee8-17f3-4d22-b7e9-46616da46780)

403 Forbidden: You don't have the rights to create a model under the namespace "WindyAle".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

In [5]:
# 업로드
print(f"\n[5/5] Hugging Face 업로드 시작 ({TARGET_REPO_ID})...")
login(token=HF_TOKEN)    
api = HfApi()

api.create_repo(repo_id=TARGET_REPO_ID, exist_ok=True, repo_type="model")

try:
    api.upload_file(
        path_or_fileobj=GGUF_FILENAME_Q4,
        path_in_repo=GGUF_FILENAME_Q4,
        repo_id=TARGET_REPO_ID,
        repo_type="model"
    )
    print("\n🎉 모든 작업이 완료되었습니다!")
    print(f"🔗 모델 링크: https://huggingface.co/{TARGET_REPO_ID}")
    
except Exception as e:
    print(f"❌ 업로드 중 오류 발생: {e}")


[5/5] Hugging Face 업로드 시작 (WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF)...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


🎉 모든 작업이 완료되었습니다!
🔗 모델 링크: https://huggingface.co/WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF
